In [1]:
import sys
import numpy as np
import math

In [ ]:
# input parameters
#number of trajectories
ntrajs=
#number of time blocks
ntblock=
#delta t
dt=
#counter index ?
block=
#very important tuning parameter for moving distribution of new function
k=

# input variable
obsts=np.empty((ntrajs,ntblock))

In [ ]:
# resampling

# compute weights
obsint=np.empty(ntrajs)
for traj in range(ntrajs):
    obsint[traj]=np.sum(obsts[traj,:])*dt
W=np.exp(k*(obsint))

# compute the normalizer and the normalized weights
R=sum(W)/ntrajs
w=W/R

# compute the number of clones generated by trajectory
nc=np.empty(ntrajs)
for traj in range(ntrajs):
    nc[traj]=math.floor(w[traj]+np.random.uniform(0,1))

# compute total number of clones and the difference with original ensemble size 
Nc=sum(nc)
dN=Nc-ntrajs

In [ ]:
'''
if the clones are less than what is needed, clone randomly among the successfull 
trajectories, proportionally to the number of clones they are originally meant 
to produce, with repetition
'''
label=np.arange(ntrajs)
if dN<0:
    traj2clone=np.empty(int(-dN))
    for traj in range(int(-dN)):    
        traj2clone[traj]=np.random.choice(label,p=nc/sum(nc))  
    for traj in range(int(-dN)):
        nc[int(traj2clone[traj])]=nc[int(traj2clone[traj])]+1
        
'''
if the clones are more than what is needed, kill randomly among the successfull 
trajectories, proportionally to the number of clones they are originally meant 
to produce, without repetition
'''

if dN>0:
    traj2kill=np.empty(int(dN))
    for traj in range(int(dN)):
        traj2kill[traj]=np.random.choice(label,p=nc/sum(nc))
        nc[int(traj2kill[traj])]=nc[int(traj2kill[traj])]-1

'''
note on the procedure above: the random selection is performed proportionally 
to the number of clones the trajectories are originally meant to produce. This 
means selecting on the population of trajectories present after the resampling: 
what is done above is a correction to the resampling.

define a label array with 0 if the trajectory survives, 1 if it is killed and 
it is meant to be substituted by a copy of a successfull trajectory
'''
killedlabel=np.empty(ntrajs)
for traj in range(ntrajs):
    if nc[traj]>0:
        killedlabel[traj]=0
    else:
        killedlabel[traj]=1


In [ ]:
'''
for each trajectory, determine the label of the trajectory whose restart will 
be used to continue the evolution. Originally this initialisation label is equal 
to the label of the trajectory itself: the trajectory continues its own evolution.
If nc is 0 (the trajectory will be killed) or 1 (the trajectory survives but does not  
generate clones) nothing is done. If nc>1 (the trajectory spawns nc-1 clones, 
in addition to continuing its own evolution), pick randomly nc-1 trajectories 
among those meant to be killed, and give them the label of the spawning trajectory.
The label array of the trajectories to be killed is updated every time, in order to
be sure to substitute all the killed trajectories by the end of the loop.  

'''
initlabel=np.arange(ntrajs)
udkilledlabel=np.empty(ntrajs)
udkilledlabel[:]=killedlabel[:]
for traj in range(ntrajs):
    if nc[traj]>1:
        for n in range(int(nc[traj])-1):
            traj2substitute=np.random.choice(label,p=udkilledlabel/sum(udkilledlabel))
            initlabel[int(traj2substitute)]=traj
            udkilledlabel[int(traj2substitute)]=0

In [ ]:
# from here on change how to save output depending on the model

# write the init files accordingly to the cloning, and copy them into the run folders
for traj in range(ntrajs):
    initfilename=expname+'_init.'+str(traj+1).zfill(4)+'.'+str(block+1).zfill(4)
    restfilename=expname+'_rest.'+str(initlabel[traj]+1).zfill(4)+'.'+str(block).zfill(4)
    copy(restexpdir+'/block_'+str(block).zfill(4)+'/'+restfilename,initexpdir+'/block_'+str(block+1).zfill(4)+'/'+initfilename)

# save all the relevant informations about the resampling                        
np.savez(resamplingexpdir+'/block_'+str(block).zfill(4)+'/'+expname+'_resampling.'+str(block).zfill(4),\
         ntrajs=ntrajs,Nc=Nc,dN=dN,R=R,W=W,w=w,nc=nc,killedlabel=killedlabel,udkilledlabel=udkilledlabel,initlabel=initlabel,k=k,dt=dt,\
         obsts=obsts,obsmean=obsmean,LAT=LAT,LON=LON,mask_world=mask_world,mask_Scandinavia=mask_Scandinavia,gparea=gparea)
